1. Imports et configuration des chemins

In [2]:
# 1. Imports et configuration des chemins

from pathlib import Path
from sqlalchemy import create_engine
from sqlalchemy import text
from sqlalchemy import event
import pandas as pd
import sqlite3
import numpy as np
import time
import gc


# Racine du projet (notebook dans /notebooks → racine = parent)
ROOT = Path("..").resolve()

# Dossiers
RAW_DIR = ROOT / "data" / "raw"
DB_DIR = ROOT / "db"

# Base SQLite de RENDU (nouveau fichier pour éviter le blocage Windows)
DB_PATH = DB_DIR / "olist_final.sqlite"

# Schéma SQL
SCHEMA_PATH = ROOT / "sql" /"ddl"/"schema_etoile.sql"

# Checks
assert RAW_DIR.exists(), f"RAW_DIR introuvable: {RAW_DIR}"
assert SCHEMA_PATH.exists(), f"SCHEMA introuvable: {SCHEMA_PATH}"
DB_DIR.mkdir(parents=True, exist_ok=True)

ROOT, RAW_DIR, DB_PATH, SCHEMA_PATH

(WindowsPath('C:/Data/Semaine-9 février 2026/olist-data-cleaning'),
 WindowsPath('C:/Data/Semaine-9 février 2026/olist-data-cleaning/data/raw'),
 WindowsPath('C:/Data/Semaine-9 février 2026/olist-data-cleaning/db/olist_final.sqlite'),
 WindowsPath('C:/Data/Semaine-9 février 2026/olist-data-cleaning/sql/ddl/schema_etoile.sql'))

2. Connexion à SQLite avec SQLAlchemy

In [3]:
# Cellule “anti-blocage Windows”

# Ferme proprement l'engine si déjà créé (évite WinError 32)
try:
    engine.dispose()
    print("engine.dispose() OK")
except Exception as e:
    print("engine pas dispo ou déjà fermé:", e)

# Force le nettoyage d'objets pouvant garder une connexion ouverte
gc.collect()

engine pas dispo ou déjà fermé: name 'engine' is not defined


0

In [4]:
# 2.0 Recréer fichier DB propre

if DB_PATH.exists():
    for i in range(5):
        try:
            DB_PATH.unlink()
            print("DB supprimée:", DB_PATH)
            break
        except PermissionError:
            print(f"DB verrouillée, retry {i+1}/5...")
            time.sleep(0.5)
    else:
        raise PermissionError(
            "La DB est verrouillée. Ferme DB Browser/VSCode SQLite viewer, "
            "puis Kernel > Restart et relance."
        )
else:
    print("DB inexistante, OK:", DB_PATH)

DB supprimée: C:\Data\Semaine-9 février 2026\olist-data-cleaning\db\olist_final.sqlite


In [5]:
# 2.1 Création de l’engine

engine = create_engine(f"sqlite:///{DB_PATH}")
print("Engine prêt.")

Engine prêt.


In [6]:
# 2.2 Activation des clés étrangères (FK)

@event.listens_for(engine, "connect")
def set_sqlite_pragma(dbapi_connection, connection_record):
    cursor = dbapi_connection.cursor()
    cursor.execute("PRAGMA foreign_keys=ON;")
    cursor.close()

In [7]:
# 2.3 Test rapide de connexion (SELECT 1)
with engine.connect() as conn:
    res = conn.execute(text("SELECT 1")).scalar()
print("Test SQL réussi ->", res)

Test SQL réussi -> 1


In [8]:
# 2.4 RESET + application du schéma SQL (schema_etoile)
schema_sql = SCHEMA_PATH.read_text(encoding="utf-8")

conn = sqlite3.connect(DB_PATH)
conn.execute("PRAGMA foreign_keys=ON;")

# On réinitialise le schéma sans supprimer le fichier
conn.executescript(schema_sql)

conn.commit()
conn.close()

print("Schéma (ré)appliqué sur:", DB_PATH)

Schéma (ré)appliqué sur: C:\Data\Semaine-9 février 2026\olist-data-cleaning\db\olist_final.sqlite


In [9]:
# 2.5 Test de vérification du schéma appliqué

with engine.connect() as conn:
    tables = conn.execute(text("""
        SELECT name FROM sqlite_master 
        WHERE type='table' 
        ORDER BY name;
    """)).fetchall()

tables

[('dim_customers',),
 ('dim_date',),
 ('dim_products',),
 ('dim_sellers',),
 ('fact_order_items',)]

3. Chargement des fichiers CSV bruts

In [10]:
# 3.1 customers
customers = pd.read_csv(RAW_DIR / "olist_customers_dataset.csv")
customers.head()

# 3.2 products
products = pd.read_csv(RAW_DIR / "olist_products_dataset.csv")
products.head()

# 3.3 sellers
sellers = pd.read_csv(RAW_DIR / "olist_sellers_dataset.csv")
sellers.head()

# 3.4 orders
orders = pd.read_csv(RAW_DIR / "olist_orders_dataset.csv")
orders.head()

# 3.5 order_items
order_items = pd.read_csv(RAW_DIR / "olist_order_items_dataset.csv")
order_items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [11]:
# 3.6 Sanity check : shapes + colonnes attendues
print("customers", customers.shape)
print("products", products.shape)
print("sellers", sellers.shape)
print("orders", orders.shape)
print("order_items", order_items.shape)

print("\norders columns:", list(orders.columns))
print("order_items columns:", list(order_items.columns))

customers (99441, 5)
products (32951, 9)
sellers (3095, 4)
orders (99441, 8)
order_items (112650, 7)

orders columns: ['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']
order_items columns: ['order_id', 'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date', 'price', 'freight_value']


4. Préparation des tables de dimensions

In [12]:
# 4.1 dim_customers : sélection colonnes + dédoublonnage PK
dim_customers = customers[[
    "customer_id", "customer_unique_id", "customer_city", "customer_state"
]].drop_duplicates(subset=["customer_id"]).copy()

print("dim_customers", dim_customers.shape)
dim_customers.head()

dim_customers (99441, 4)


,customer_id,customer_unique_id,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,campinas,SP


In [13]:
# 4.2 dim_products : sélection colonnes + dédoublonnage PK

dim_products = products[[
    "product_id",
    "product_category_name",
    "product_name_lenght",
    "product_description_lenght",
    "product_photos_qty",
    "product_weight_g",
    "product_length_cm",
    "product_height_cm",
    "product_width_cm"
]].drop_duplicates(subset=["product_id"]).copy()

print("dim_products", dim_products.shape)
dim_products.head()


dim_products (32951, 9)


,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [14]:
# 4.3 dim_sellers : sélection colonnes + dédoublonnage PK
dim_sellers = sellers[[
    "seller_id", 
    "seller_city", 
    "seller_state"
]].drop_duplicates(subset=["seller_id"]).copy()

print("dim_sellers", dim_sellers.shape)
dim_sellers.head()

dim_sellers (3095, 3)


,seller_id,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,braganca paulista,SP


In [15]:
# 4.3 dim_sellers : sélection colonnes + dédoublonnage PK
dim_sellers = sellers[[
    "seller_id", "seller_city", "seller_state"
]].drop_duplicates(subset=["seller_id"]).copy()

print("dim_sellers", dim_sellers.shape)
dim_sellers.head()

dim_sellers (3095, 3)


,seller_id,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,braganca paulista,SP


In [16]:
# 4.4 dim_date [YYYY-MM-DD] : extraction des dates depuis orders ---
# 1) Parsing dates
date_cols = [
    "order_purchase_timestamp",
    "order_delivered_customer_date",
    "order_estimated_delivery_date",
]

tmp = orders[date_cols].copy()
for c in date_cols:
    tmp[c] = pd.to_datetime(tmp[c], errors="coerce")

# 2) Collecte de toutes les dates [YYYY-MM-DD strings]
all_dates = pd.concat([tmp[c].dropna().dt.date for c in date_cols]).drop_duplicates()

dim_date = pd.DataFrame({"date_id": all_dates.astype(str)})
dim_date["year"] = pd.to_datetime(dim_date["date_id"]).dt.year
dim_date["month"] = pd.to_datetime(dim_date["date_id"]).dt.month
dim_date["day"] = pd.to_datetime(dim_date["date_id"]).dt.day

dim_date = dim_date.drop_duplicates(subset=["date_id"]).sort_values("date_id").reset_index(drop=True)

print("dim_date", dim_date.shape)
dim_date.head()

dim_date (710, 4)


,date_id,year,month,day
0,2016-09-04,2016,9,4
1,2016-09-05,2016,9,5
2,2016-09-13,2016,9,13
3,2016-09-15,2016,9,15
4,2016-09-30,2016,9,30


In [17]:
# 4.5 Aperçu final des dimensions (head + shapes)
for name, df in [
    ("dim_customers", dim_customers),
    ("dim_products", dim_products),
    ("dim_sellers", dim_sellers),
    ("dim_date", dim_date),
]:
    print(name, df.shape)

dim_customers (99441, 4)
dim_products (32951, 9)
dim_sellers (3095, 3)
dim_date (710, 4)


5. Contrôles qualité avant insertion (vérifier clés primaires vides, doublons, colonnes mal typées)

In [18]:
# 5.1 Fonction utilitaire check_pk(df, pk_col, name)
def check_pk(df, pk_col, name):
    assert df[pk_col].notna().all(), f"{name}: PK null"
    assert df[pk_col].is_unique, f"{name}: PK dupliquée"
    print(f"{name}: PK OK")

In [19]:
# 5.2 Contrôles PK + doublons (dimensions)
check_pk(dim_customers, "customer_id", "dim_customers")
check_pk(dim_products, "product_id", "dim_products")
check_pk(dim_sellers, "seller_id", "dim_sellers")
check_pk(dim_date, "date_id", "dim_date")

dim_customers: PK OK
dim_products: PK OK
dim_sellers: PK OK
dim_date: PK OK


In [20]:
# 5.3 Contrôles types simples 
# colonnes numériques
assert pd.api.types.is_numeric_dtype(order_items["price"])
assert pd.api.types.is_numeric_dtype(order_items["freight_value"])
print("Types numériques OK")

Types numériques OK


In [21]:
# 5.4 Préparation de la TABLE DE FAITS (construction)
orders_dates = orders[[
    "order_id",
    "customer_id",
    "order_purchase_timestamp",
    "order_delivered_customer_date",
    "order_estimated_delivery_date"
]].copy()

orders_dates["purchase_date_id"] = pd.to_datetime(
    orders_dates["order_purchase_timestamp"], errors="coerce"
).dt.date.astype("string")

orders_dates["delivered_date_id"] = pd.to_datetime(
    orders_dates["order_delivered_customer_date"], errors="coerce"
).dt.date.astype("string")

orders_dates["estimated_date_id"] = pd.to_datetime(
    orders_dates["order_estimated_delivery_date"], errors="coerce"
).dt.date.astype("string")

fact = order_items.merge(
    orders_dates[[
        "order_id",
        "customer_id",
        "purchase_date_id",
        "delivered_date_id",
        "estimated_date_id"
    ]],
    on="order_id",
    how="left"
)

fact_order_items = fact[[
    "order_id",
    "order_item_id",
    "customer_id",
    "product_id",
    "seller_id",
    "purchase_date_id",
    "delivered_date_id",
    "estimated_date_id",
    "price",
    "freight_value"
]].copy()

print("fact_order_items", fact_order_items.shape)
fact_order_items.head()

fact_order_items (112650, 10)


,order_id,order_item_id,customer_id,product_id,seller_id,purchase_date_id,delivered_date_id,estimated_date_id,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,3ce436f183e68e07877b285a838db11a,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-13,2017-09-20,2017-09-29,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,f6dd3ec061db4e3987629fe6b26e5cce,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-04-26,2017-05-12,2017-05-15,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,6489ae5e4333f3693df5ad4372dab6d3,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-14,2018-01-22,2018-02-05,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,d4eb9395c8c0431ee92fce09860c5a06,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-08,2018-08-14,2018-08-20,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,58dbd0b2d70206bf40e62cd34e84d795,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-04,2017-03-01,2017-03-17,199.90,18.14


In [22]:
# 5.5 Contrôles PK (fact) : (order_id, order_item_id) unique + non null
# PK composite : (order_id, order_item_id)
assert fact_order_items["order_id"].notna().all(), "fact_order_items: order_id null"
assert fact_order_items["order_item_id"].notna().all(), "fact_order_items: order_item_id null"

dup_pk = fact_order_items.duplicated(subset=["order_id", "order_item_id"]).sum()
assert dup_pk == 0, f"fact_order_items: PK dupliquée -> {dup_pk}"

print("fact_order_items: PK OK")

fact_order_items: PK OK


In [23]:
# 5.6 Contrôles “FK de base” avant insertion (valeurs existantes dans dims)

# Vérifie que les IDs présents dans la TABLE DE FAITS existent dans les dimensions préparées
missing_customers = (~fact_order_items["customer_id"].isin(dim_customers["customer_id"])).mean()
missing_products  = (~fact_order_items["product_id"].isin(dim_products["product_id"])).mean()
missing_sellers   = (~fact_order_items["seller_id"].isin(dim_sellers["seller_id"])).mean()
missing_purchase_dates = (~fact_order_items["purchase_date_id"].isin(dim_date["date_id"])).mean()

print("missing customer_id %:", round(missing_customers * 100, 4))
print("missing product_id  %:", round(missing_products * 100, 4))
print("missing seller_id   %:", round(missing_sellers * 100, 4))
print("missing purchase_date_id %:", round(missing_purchase_dates * 100, 4))

missing customer_id %: 0.0
missing product_id  %: 0.0
missing seller_id   %: 0.0
missing purchase_date_id %: 0.0


6. Insertion dans SQLite

In [24]:
# 6.1 Epuration des tables (DELETE) en ordre correct
with engine.begin() as conn:
    
    conn.execute(text("DELETE FROM fact_order_items;"))
    conn.execute(text("DELETE FROM dim_date;"))
    conn.execute(text("DELETE FROM dim_customers;"))
    conn.execute(text("DELETE FROM dim_products;"))
    conn.execute(text("DELETE FROM dim_sellers;"))

print("Tables vidées")

Tables vidées


In [25]:
# 6.2 FIX dim_date et insertion des dimensions (to_sql append)

# 0) Reset du notebook
with engine.begin() as conn:
    conn.execute(text("DELETE FROM fact_order_items;"))
    conn.execute(text("DELETE FROM dim_date;"))
    conn.execute(text("DELETE FROM dim_customers;"))
    conn.execute(text("DELETE FROM dim_products;"))
    conn.execute(text("DELETE FROM dim_sellers;"))

# 1) Insert dims simples
dim_customers.to_sql("dim_customers", engine, if_exists="append", index=False)
dim_products.to_sql("dim_products", engine, if_exists="append", index=False)
dim_sellers.to_sql("dim_sellers", engine, if_exists="append", index=False)

# 2) FIX dim_date (crée dim_date_fixed ici, donc jamais NameError)
d = pd.to_datetime(dim_date["date"], errors="coerce") if "date" in dim_date.columns else pd.to_datetime(dim_date["date_id"], errors="coerce")
d = d.dt.normalize().dropna()

dim_date_fixed = pd.DataFrame({"date": d.drop_duplicates()})
dim_date_fixed["date_id"] = dim_date_fixed["date"].dt.strftime("%Y%m%d").astype("int64")
dim_date_fixed["year"] = dim_date_fixed["date"].dt.year.astype("int64")
dim_date_fixed["month"] = dim_date_fixed["date"].dt.month.astype("int64")
dim_date_fixed["day"] = dim_date_fixed["date"].dt.day.astype("int64")

# 3) Aligner sur les colonnes de la table SQLite
with engine.connect() as conn:
    cols_db = [c[1] for c in conn.execute(text("PRAGMA table_info(dim_date);"))]

dim_date_fixed = dim_date_fixed[[c for c in cols_db if c in dim_date_fixed.columns]]
dim_date_fixed = dim_date_fixed.drop_duplicates(subset=["date_id"]).sort_values("date_id").reset_index(drop=True)

# 4) Insert dim_date
dim_date_fixed.to_sql("dim_date", engine, if_exists="append", index=False)

print("Dimensions insérées")

Dimensions insérées


In [26]:
# 6.3 Conversion 3 colonnes en YYYYMMDD
for c in ["purchase_date_id", "delivered_date_id", "estimated_date_id"]:
    fact_order_items[c] = pd.to_datetime(fact_order_items[c], errors="coerce").dt.strftime("%Y%m%d")
    fact_order_items[c] = pd.to_numeric(fact_order_items[c], errors="coerce").astype("Int64")

In [27]:
# 6.4 Insertion fact_order_items (to_sql append)

with engine.begin() as conn:
    conn.execute(text("DELETE FROM fact_order_items;"))

fact_order_items.to_sql("fact_order_items", engine, if_exists="append", index=False)
print("table de faits insérée")

table de faits insérée


In [28]:
# 6.5 Vérification finale des FK

with engine.connect() as conn:
    missing_dates = conn.execute(text("""
        SELECT COUNT(*)
        FROM fact_order_items f
        LEFT JOIN dim_date d ON f.purchase_date_id = d.date_id
        WHERE d.date_id IS NULL;
    """)).scalar()

print("Missing purchase_date FK:", missing_dates)


Missing purchase_date FK: 0


In [29]:
# 6.6 Contrôle d’erreurs d’insertion 

    # - Capturer une erreur SQLite (clé dupliquée, FK, type…)
    # - Afficher un message clair
    # - Éviter un arrêt inopiné du notebook  

with engine.connect() as conn:
    n = conn.execute(text("SELECT COUNT(*) FROM fact_order_items")).scalar()

if n > 0:
    print(f"fact_order_items contient déjà {n} lignes -> insertion ignorée (évite doublons)")
else:
    try:
        fact_order_items.to_sql("fact_order_items", engine, if_exists="append", index=False)
        print("Insertion fact_order_items réussie")
    except Exception as e:
        print("Erreur lors de l’insertion fact_order_items")
        print(type(e).__name__)
        print(e)

fact_order_items contient déjà 112650 lignes -> insertion ignorée (évite doublons)


7. Contrôles de cohérence après insertion (compter les lignes et vérifier les correspondances FK de base)

In [30]:
# 7.1 Comptage lignes par table (SELECT COUNT(*))
with engine.connect() as conn:
    for t in ["dim_customers", "dim_products", "dim_sellers", "dim_date", "fact_order_items"]:
        n = conn.execute(text(f"SELECT COUNT(*) FROM {t}")).scalar()
        print(t, n)

dim_customers 99441
dim_products 32951
dim_sellers 3095
dim_date 710
fact_order_items 112650


# 7.2 Check doublons en base (vérifier unicité des PK après insertion)

In [31]:
# 1) Check doublons PK sur dim_date
with engine.connect() as conn:
    n_dup = conn.execute(text("""
        SELECT COUNT(*) FROM (
            SELECT date_id
            FROM dim_date
            GROUP BY date_id
            HAVING COUNT(*) > 1
        )
    """)).scalar()

print("Doublons PK dim_date:", n_dup)

Doublons PK dim_date: 0


In [32]:
# 2) Check doublons PK sur les 3 autres dimensions : dim_customers, dim_products, dim_sellers
with engine.connect() as conn:
    n_dup_customers = conn.execute(text("""
        SELECT COUNT(*) FROM (
            SELECT customer_id FROM dim_customers
            GROUP BY customer_id
            HAVING COUNT(*) > 1
        )
    """)).scalar()

    n_dup_products = conn.execute(text("""
        SELECT COUNT(*) FROM (
            SELECT product_id FROM dim_products
            GROUP BY product_id
            HAVING COUNT(*) > 1
        )
    """)).scalar()

    n_dup_sellers = conn.execute(text("""
        SELECT COUNT(*) FROM (
            SELECT seller_id FROM dim_sellers
            GROUP BY seller_id
            HAVING COUNT(*) > 1
        )
    """)).scalar()

print("Doublons PK dim_customers:", n_dup_customers)
print("Doublons PK dim_products :", n_dup_products)
print("Doublons PK dim_sellers  :", n_dup_sellers)

Doublons PK dim_customers: 0
Doublons PK dim_products : 0
Doublons PK dim_sellers  : 0


In [33]:
# 3) Check doublons PK composite sur la TABLE DE FAITS
with engine.connect() as conn:
    n_dup_fact = conn.execute(text("""
        SELECT COUNT(*) FROM (
            SELECT order_id, order_item_id
            FROM fact_order_items
            GROUP BY order_id, order_item_id
            HAVING COUNT(*) > 1
        )
    """)).scalar()

print("Doublons PK fact_order_items:", n_dup_fact)

Doublons PK fact_order_items: 0


In [34]:
# 7.3 Check FK “simple”, post-insertion (LEFT JOIN / IS NULL)
with engine.connect() as conn:
    n_bad = conn.execute(text("""
        SELECT COUNT(*) 
        FROM fact_order_items f
        LEFT JOIN dim_customers c ON f.customer_id = c.customer_id
        WHERE c.customer_id IS NULL
    """)).scalar()
print("FK fact->customers manquantes:", n_bad)

FK fact->customers manquantes: 0


In [35]:
# 7.4 FK checks post-insertion (products + sellers)
with engine.connect() as conn:
    bad_prod = conn.execute(text("""
        SELECT COUNT(*) 
        FROM fact_order_items f
        LEFT JOIN dim_products p ON f.product_id = p.product_id
        WHERE p.product_id IS NULL
    """)).scalar()

    bad_sell = conn.execute(text("""
        SELECT COUNT(*) 
        FROM fact_order_items f
        LEFT JOIN dim_sellers s ON f.seller_id = s.seller_id
        WHERE s.seller_id IS NULL
    """)).scalar()

print("FK fact->products manquantes:", bad_prod)
print("FK fact->sellers manquantes :", bad_sell)

FK fact->products manquantes: 0
FK fact->sellers manquantes : 0
